In [1]:
%pip install --quiet --upgrade numpy soundfile IPython


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import numpy as np, soundfile as sf
from math import pi, cos
from IPython.display import Audio, display, HTML

# ---------- modal frequency tables --------------------------------
BESSEL_RATIOS = [
    1.000, 1.593, 2.136, 2.650, 3.142, 3.615, 4.072, 4.515,
    4.946, 5.368, 5.783, 6.191, 6.592, 6.988, 7.378,
    7.764, 8.145, 8.521, 8.894, 9.263
]

def modal_frequencies(f1, n, kind="bessel"):
    """
    kind='bessel'  : circular membrane Bessel ratios
    kind='harmonic': 1×,2×,3×,... harmonic partials
    """
    if kind == "harmonic":
        return [f1 * (m + 1) for m in range(n)]
    else:  # bessel default
        return [f1 * r for r in BESSEL_RATIOS[:n]]

def excite(ms=5, fs=48_000):
    n  = int(fs * ms * 1e-3)
    noise = 0.3 * np.random.randn(n)
    env   = np.exp(-np.arange(n) / fs * 800)   # 1.25-ms decay
    return noise * env

class ModalResonator:
    def __init__(self, f0, t60, fs):
        self.fs  = fs
        self.t60 = t60
        self._set_coeffs(f0, t60)
        self.y1 = self.y2 = 0.0

    def _set_coeffs(self, f0, t60):
        r  = 10 ** (-3 / (t60 * self.fs))
        w0 = 2 * pi * f0 / self.fs
        self.a1 = -2 * r * cos(w0)
        self.a2 =  r * r
        self.g  = 1.0         # unity peak

    def process(self, x):
        y = self.g * x - self.a1*self.y1 - self.a2*self.y2
        self.y2, self.y1 = self.y1, y
        return y

# ---------- ResonatorBank -----------------------------------------
class ResonatorBank:
    def __init__(self, f1=140, n=16, t60=3.0, slope=-9, fs=48_000,
                 ring_gain=0.18, kind="bessel"):
        self.ring_gain = ring_gain
        self.modes = []
        for f in modal_frequencies(f1, n, kind=kind):
            t = t60 * 10**((slope/6)*np.log2(f/f1))
            self.modes.append(ModalResonator(f, t, fs))

    def tail_seconds(self, cut_dB=90, safety=1.05):
        t60_max = max(m.t60 for m in self.modes)
        return safety * (cut_dB/60) * t60_max

    def process(self, x):
        return self.ring_gain * sum(m.process(x) for m in self.modes)

# ---------- render helper -----------------------------------------
def render_drum(filename, bank, dry_mix=0.03,
                fs=48_000, cut_dB=90):
    tail = bank.tail_seconds(cut_dB=cut_dB)
    inp  = np.concatenate([excite(5, fs),
                           np.zeros(int(fs*tail))])

    out = np.zeros_like(inp)
    for n, x in enumerate(inp):
        out[n] = dry_mix * x + bank.process(x)

    # normalise & save
    if (pk := np.max(np.abs(out))) > 0:
        out /= pk
    sf.write(filename, np.clip(out, -1, 1), fs, subtype="PCM_16")
    print(f"Wrote {filename}  •  {len(out)/fs:.2f} s")

# ---------- common parameters -------------------------------------
fs        = 48_000
f1        = 140       
n_modes   = 64
t60_base  = 5.0
ring_gain = 0.18
slope     = -7

# ---------- render both versions ----------------------------------
bank_bessel = ResonatorBank(f1, n_modes, t60_base, slope, fs,
                            ring_gain, kind="bessel")
render_drum("drum_bessel.wav", bank_bessel, fs=fs)

bank_harm   = ResonatorBank(f1, n_modes, t60_base, slope, fs,
                            ring_gain, kind="harmonic")
render_drum("drum_harmonic.wav", bank_harm, fs=fs)

# ---------- inline A/B player -------------------------------------
display(HTML("<b>Bessel-mode drum</b>"))
display(Audio("drum_bessel.wav"))
display(HTML("<b>Harmonic drum</b>"))
display(Audio("drum_harmonic.wav"))

Wrote drum_bessel.wav  •  7.88 s
Wrote drum_harmonic.wav  •  7.88 s


In [3]:

f1_body    = 500       
f1_rim     = 500        
n_body     = 40
n_rim      = 18
t60_body   = 5.0
t60_rim    = 2.0
slope_body = -1
slope_rim  = -6
ring_body  = 0.3
ring_rim   = 0.22
rim_mix    = 1       # 0 = old sound, 1 = only rim bank
dry_mix    = 0.03

# --- build banks ---------------------------------------------------
body_bank = ResonatorBank(
    f1=f1_body, n=n_body, t60=t60_body,
    slope=slope_body, fs=fs, ring_gain=ring_body,
)

rim_bank = ResonatorBank(
    f1=f1_rim, n=n_rim, t60=t60_rim,
    slope=slope_rim, fs=fs, ring_gain=ring_rim,
)

# tail long enough for the slower bank
tail_secs = max(body_bank.tail_seconds(), rim_bank.tail_seconds())
inp = np.concatenate([excite(5, fs),
                      np.zeros(int(fs * tail_secs))])

out = np.zeros_like(inp)
for n, x in enumerate(inp):
    y_body = body_bank.process(x)
    y_rim  = rim_bank.process(x)
    out[n] = dry_mix * x + (1 - rim_mix) * y_body + rim_mix * y_rim

# normalise & save
out /= np.max(np.abs(out)) + 1e-12
sf.write("drum_dualbank.wav", np.clip(out, -1, 1), fs, subtype="PCM_16")
print("Wrote drum_dualbank.wav")
display(Audio("drum_dualbank.wav"))

Wrote drum_dualbank.wav
